In [ ]:
import nibabel as nib
import numpy as np

In [ ]:
# A function that tests the number of voxels that are useful in a DWI.
path = "../Data-NODDI/s01_still/s01_still_NDI.nii"
ODI_img = nib.load(path)
ODI_data = ODI_img.get_fdata()
non_zero = 0
zeros = 0
for ele in ODI_data:
    for el in ele:
        for e in el:
            if e != 0.0:
                non_zero += 1
            else:
                zeros += 1
print(non_zero)
print(zeros)
print("useful voxels ratio: "+str(non_zero/(non_zero + zeros)))

In [ ]:
# packages that helps to evaluate the model performace
from utils import calc_RMSE, calc_psnr, calc_ssim
def evaluate_model(pred, target, mask):
    rmse = calc_RMSE(pred, target, mask)
    psnr = calc_psnr(pred, target)
    ssim = calc_ssim(pred, target)
    return(rmse, psnr, ssim)

In [ ]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [ ]:
"""
packages that generate train/test dataset
"""
from FormatData import generate_data, parser as data_parser

In [ ]:
"""
packages that trains network
"""
from Training import train_network
from utils.model import parser as model_parser

In [ ]:
"""
packages that test network
"""
from Testing import test_model

In [ ]:
"""
packages that plot graphs
"""
import matplotlib.pyplot as plt
%matplotlib inline
def plot_loss(cmd):
    """
    A function that used to plot the loss curve for the trained network.
    Args:
        cmd: String, the command line in the terminal
    """
    args = model_parser().parse_args(cmd.split())
    history = train_network(args)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

def show_slices(slices):
   """ Function to display row of image slices """
   fig, axes = plt.subplots(1, len(slices))
   for i, slice in enumerate(slices):
       axes[i].imshow(slice.T, cmap="gray", origin="lower")

<h3>Experiment 1. We want to discover the number of hidden acquired to produce promising each NODDI measure.</h3>

<strong>Training subject: s01_still<br/>

Test subject: s02_still<br/>

Network; ANN<br/>

Outcome: NDI<br/>


The training data is stored in datasets/data/s01_still-96-first-1d.mat<br/>

The training label is stored in datasets/label/s01_still-NDI-96-first-1d.mat<br/>

The testing data is stored in datasets/data/s02_still-96-first.mat<br/>

The testing label is stored in datasets/label/s02_still-NDI-96-first.mat</strong>


In [ ]:
"""
Generate training dataset NDI for ANN.
Using all of the DWI. 96. as training size.
"""
cmd = "--subjects s01_still --nDWI 96 --fc1d_train --label_type N"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Generate testing dataset. 
The testing dataset does NOT need to be processed. 
The testing samples are DWI images of size (85,85,50,96).
The testing samples can be used for ANY network in this experiment.
"""
cmd = "--subjects s02_still --nDWI 96 --label_type N --test"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Using nib to fetch the  data, used for evaluation
"""
s02_NDI_path = '../Data-NODDI/s02_still/s02_still_NDI.nii'
s02_mask_path = '../Net-DTI/datasets/mask/mask_s02_still.nii'
s02_NDI_img = nib.load(s02_NDI_path)
s02_NDI_img_data = s02_NDI_img.get_fdata()
s02_mask_img = nib.load(s02_mask_path)
s02_mask_img_data = s02_mask_img.get_fdata()

<strong>Train ANNs with different number of layers.<br/>
Herein, at most 6-hidden-layer ANNs are trained.<br/>
Loss curve for each ANN is plotted.<br/>
The test result is also generated -- evaluation method used is RMSE between pred and reference.<br/>
The reference image and the genereated image are displayed side by side</strong>

<strong>1 layer</strong>

In [ ]:
# 1 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 1 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 1 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 1 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-fc1d-1layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>2 layer</strong>

In [ ]:
# 2 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 2 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 2 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 2 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-fc1d-2layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>3 layer</strong>

In [ ]:
# 3 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 3 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 3 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 3 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-fc1d-3layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>4 layer</strong>

In [ ]:
# 4 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 4 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 4 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 4 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-fc1d-4layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>5 layer</strong>

In [ ]:
# 5 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 5 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 5 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 5 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-fc1d-5layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>6 layer</strong>

In [ ]:
# 6 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 6 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 6 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 6 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-fc1d-6layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>Training subject: s01_still</br>
Test subject: s02_still</br>
Network; ANN</br>
Outcome: ODI</br>

The training data is stored in datasets/data/s01_still-96-first-1d.mat</br>
The training label is stored in datasets/label/s01_still-ODI-96-first-1d.mat</br>
The testing data is stored in datasets/data/s02_still-96-first.mat</br>
The testing label is stored in datasets/label/s02_still-ODI-96-first.mat</strong>

In [ ]:
"""
Generate training dataset ODI for ANN.
Using all of the DWI. 96. as training size.
"""
cmd = "--subjects s01_still --nDWI 96 --fc1d_train --label_type O"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Generate testing dataset. 
The testing dataset does NOT need to be processed. 
The testing samples are DWI images of size (85,85,50,96).
The testing samples can be used for ANY network in this experiment.
"""
cmd = "--subjects s02_still --nDWI 96 --label_type O --test"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Using nib to fetch the  data, used for evaluation
"""
s02_ODI_path = '../Data-NODDI/s02_still/s02_still_ODI.nii'
s02_mask_path = '../Net-DTI/datasets/mask/mask_s02_still.nii'
s02_ODI_img = nib.load(s02_ODI_path)
s02_ODI_img_data = s02_ODI_img.get_fdata()
s02_mask_img = nib.load(s02_mask_path)
s02_mask_img_data = s02_mask_img.get_fdata()

<strong>1 layer</strong>

In [ ]:
# 1 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 1 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 1 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 1 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-fc1d-1layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>2 layer</strong>

In [ ]:
# 2 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 2 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 2 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 2 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-fc1d-2layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>3 layer</strong>

In [ ]:
# 3 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 3 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 3 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 3 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-fc1d-3layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>4 layer</strong>

In [ ]:
# 4 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 4 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 4 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 4 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-fc1d-4layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>5 layer</strong>

In [ ]:
# 5 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 5 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 5 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 5 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-fc1d-5layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>6 layer</strong>

In [ ]:
# 6 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 6 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 6 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 6 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-fc1d-6layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>Training subject: s01_still</br>
Test subject: s02_still</br>
Network; ANN</br>
Outcome: FWF</br>

The training data is stored in datasets/data/s01_still-96-first-1d.mat</br>
The training label is stored in datasets/label/s01_still-FWF-96-first-1d.mat</br>
The testing data is stored in datasets/data/s02_still-96-first.mat</br>
THe testing label is stored in datasets/label/s02_still-FWF-96-first.mat</strong>

In [ ]:
"""
Generate training dataset FWF for ANN.
Using all of the DWI. 96. as training size.
"""
cmd = "--subjects s01_still --nDWI 96 --fc1d_train --label_type F"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Generate testing dataset. 
The testing dataset does NOT need to be processed. 
The testing samples are DWI images of size (85,85,50,96).
The testing samples can be used for ANY network in this experiment.
"""
cmd = "--subjects s02_still --nDWI 96 --label_type F --test"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Using nib to fetch the  data, used for evaluation
"""
s02_FWF_path = '../Data-NODDI/s02_still/s02_still_FWF.nii'
s02_mask_path = '../Net-DTI/datasets/mask/mask_s02_still.nii'
s02_FWF_img = nib.load(s02_FWF_path)
s02_FWF_img_data = s02_FWF_img.get_fdata()
s02_mask_img = nib.load(s02_mask_path)
s02_mask_img_data = s02_mask_img.get_fdata()

<strong>1 layer</strong>

In [ ]:
# 1 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 1 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 1 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 1 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-fc1d-1layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_FWF_img_data[26, :, :]
ref_slice_1 = s02_FWF_img_data[:, 30, :]
ref_slice_2 = s02_FWF_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FWF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>2 layer</strong>

In [ ]:
# 2 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 2 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 2 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 2 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-fc1d-2layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_FWF_img_data[26, :, :]
ref_slice_1 = s02_FWF_img_data[:, 30, :]
ref_slice_2 = s02_FWF_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FWF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>3 layer</strong>

In [ ]:
# 3 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 3 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 3 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 3 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-fc1d-3layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_FWF_img_data[26, :, :]
ref_slice_1 = s02_FWF_img_data[:, 30, :]
ref_slice_2 = s02_FWF_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FWF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>4 layer</strong>

In [ ]:
# 4 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 4 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 4 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 4 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-fc1d-4layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)
# evaluate the model# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-fc1d-3layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()
# visualise both ref img and pred img
ref_slice_0 = s02_FWF_img_data[26, :, :]
ref_slice_1 = s02_FWF_img_data[:, 30, :]
ref_slice_2 = s02_FWF_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FWF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>5 layer</strong>

In [ ]:
# 5 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 5 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 5 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 5 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-fc1d-5layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_FWF_img_data[26, :, :]
ref_slice_1 = s02_FWF_img_data[:, 30, :]
ref_slice_2 = s02_FWF_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FWF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>6 layer</strong>

In [ ]:
# 6 layer training
cmd = "--train_subjects s01_still --DWI 96 --model fc1d --layer 6 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 6 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model fc1d --layer 6 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-fc1d-6layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_FWF_img_data[26, :, :]
ref_slice_1 = s02_FWF_img_data[:, 30, :]
ref_slice_2 = s02_FWF_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FWF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>Training subject: s01_still<br/>

Test subject: s02_still<br/>

Network; 2D-CNN<br/>

Outcome: NDI<br/>


The training data is stored in datasets/data/s01_still-96-first-2d.mat<br/>

The training label is stored in datasets/label/s01_still-NDI-96-first-2d.mat<br/>

The testing data is stored in datasets/data/s02_still-96-first.mat<br/>

The testing label is stored in datasets/label/s02_still-NDI-96-first.mat</strong>


In [ ]:
"""
Generate training dataset NDI for 2D CNN.
Using all of the DWI. 96. as training size.
"""
cmd = "--subjects s01_still --nDWI 96 --conv2d_train --label_type N"
args = data_parser().parse_args(cmd.split())
generate_data(args)

<strong>Train 2D CNNs with different number of layers.<br/>
Herein, at most 6-hidden-layer 2D CNNs are trained.<br/>
Loss curve for each 2D CNN is plotted.<br/>
The test result is also generated -- evaluation method used is RMSE between pred and reference.<br/>
The reference image and the genereated image are displayed side by side</strong>

<strong>1 layer</strong>

In [ ]:
# 1 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 1 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 1 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 1 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-conv2d-1layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>2 layer</strong>

In [ ]:
# 2 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 2 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 2 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 2 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-conv2d-2layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>3 layer</strong>

In [ ]:
# 3 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 3 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 3 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 3 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-conv2d-3layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>4 layer</strong>

In [ ]:
# 4 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 4 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 4 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 4 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-conv2d-4layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>5 layer</strong>

In [ ]:
# 5 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 5 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 5 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 5 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-conv2d-5layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>6 layer</strong>

In [ ]:
# 6 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 6 --train --label_type N"
plot_loss(cmd)

In [ ]:
# 6 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 6 --label_type N"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-NDI-96-first-conv2d-6layer.nii'
pred_NDI_img = nib.load(pred_path)
pred_NDI_img_data = pred_NDI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_NDI_img_data, s02_NDI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_NDI_img_data[26, :, :]
ref_slice_1 = s02_NDI_img_data[:, 30, :]
ref_slice_2 = s02_NDI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_NDI reference image")

pred_slice_0 = pred_NDI_img_data[26, :, :]
pred_slice_1 = pred_NDI_img_data[:, 30, :]
pred_slice_2 = pred_NDI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_NDI predicted image")

<strong>Training subject: s01_still<br/>

Test subject: s02_still<br/>

Network; 2D-CNN<br/>

Outcome: ODI<br/>


The training data is stored in datasets/data/s01_still-96-first-2d.mat<br/>

The training label is stored in datasets/label/s01_still-ODI-96-first-2d.mat<br/>

The testing data is stored in datasets/data/s02_still-96-first.mat<br/>

The testing label is stored in datasets/label/s02_still-ODI-96-first.mat</strong>

In [ ]:
"""
Generate training dataset NDI for 2D CNN.
Using all of the DWI. 96. as training size.
"""
cmd = "--subjects s01_still --nDWI 96 --conv2d_train --label_type O"
args = data_parser().parse_args(cmd.split())
generate_data(args)

<strong>1 layer</strong>

In [ ]:
# 1 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 1 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 1 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 1 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-conv2d-1layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>2 layer</strong>

In [ ]:
# 2 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 2 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 2 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 2 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-conv2d-2layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>3 layer</strong>

In [ ]:
# 3 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 3 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 3 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 3 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-conv2d-3layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>4 layer</strong>

In [ ]:
# 4 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 4 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 4 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 4 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-conv2d-4layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>5 layer</strong>

In [ ]:
# 5 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 5 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 5 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 5 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-conv2d-5layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>6 layer</strong>

In [ ]:
# 6 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 6 --train --label_type O"
plot_loss(cmd)

In [ ]:
# 6 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 6 --label_type O"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-ODI-96-first-conv2d-6layer.nii'
pred_ODI_img = nib.load(pred_path)
pred_ODI_img_data = pred_ODI_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_ODI_img_data, s02_ODI_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_ODI reference image")

pred_slice_0 = pred_ODI_img_data[26, :, :]
pred_slice_1 = pred_ODI_img_data[:, 30, :]
pred_slice_2 = pred_ODI_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_ODI predicted image")

<strong>Training subject: s01_still<br/>

Test subject: s02_still<br/>

Network; 2D-CNN<br/>

Outcome: FWF<br/>


The training data is stored in datasets/data/s01_still-96-first-2d.mat<br/>

The training label is stored in datasets/label/s01_still-FWF-96-first-2d.mat<br/>

The testing data is stored in datasets/data/s02_still-96-first.mat<br/>

The testing label is stored in datasets/label/s02_still-FWF-96-first.mat</strong>

In [ ]:
"""
Generate training dataset FWF for 2D CNN.
Using all of the DWI. 96. as training size.
"""
cmd = "--subjects s01_still --nDWI 96 --conv2d_train --label_type F"
args = data_parser().parse_args(cmd.split())
generate_data(args)

<strong>1 layer</strong>

In [ ]:
# 1 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 1 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 1 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 1 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-conv2d-1layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FwF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>2 layer</strong>

In [ ]:
# 2 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 2 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 2 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 2 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-conv2d-2layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FwF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>3 layer</strong>

In [ ]:
# 3 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 3 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 3 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 3 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-conv2d-3layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FwF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>4 layer</strong>

In [ ]:
# 4 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 4 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 4 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 4 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-conv2d-4layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FwF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>5 layer</strong>

In [ ]:
# 5 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 5 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 5 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 5 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-conv2d-5layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FwF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")

<strong>6 layer</strong>

In [ ]:
# 6 layer training
cmd = "--train_subjects s01_still --DWI 96 --model conv2d --layer 6 --train --label_type F"
plot_loss(cmd)

In [ ]:
# 6 layer testing
cmd = "--test_subjects s02_still --DWI 96 --model conv2d --layer 6 --label_type F"
args = model_parser().parse_args(cmd.split())
test_model(args)

In [ ]:
# load the reference img and pred img
pred_path = '../Net-DTI/nii/s02_still-FWF-96-first-conv2d-6layer.nii'
pred_FWF_img = nib.load(pred_path)
pred_FWF_img_data = pred_FWF_img.get_fdata()

# evaluate the model
eval_re = evaluate_model(pred_FWF_img_data, s02_FWF_img_data, s02_mask_img_data)
print(eval_re)

# visualise both ref img and pred img
ref_slice_0 = s02_ODI_img_data[26, :, :]
ref_slice_1 = s02_ODI_img_data[:, 30, :]
ref_slice_2 = s02_ODI_img_data[:, :, 16]
show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
plt.suptitle("Center slices for s02_still_FwF reference image")

pred_slice_0 = pred_FWF_img_data[26, :, :]
pred_slice_1 = pred_FWF_img_data[:, 30, :]
pred_slice_2 = pred_FWF_img_data[:, :, 16]
show_slices([pred_slice_0, pred_slice_1, pred_slice_2])
plt.suptitle("Center slices for s02_still_FWF predicted image")